In [0]:
# Change directory to VSCode workspace root so that relative path loads work correctly. Turn this addition off with the DataScience.changeDirOnImportExport setting
# ms-python.python added
import os
try:
	os.chdir(os.path.join(os.getcwd(), '..'))
	print(os.getcwd())
except:
	pass


In [1]:
import recoengi
import recoengi.cf as cf

import pandas as pd
from scipy import sparse
from sklearn import metrics
import pickle
import pkg_resources
import logging
logging.basicConfig(level=logging.DEBUG, format="%(levelname)s %(asctime)s %(message)s")



In [2]:
dtf_train = pickle.load(open(pkg_resources.resource_filename('recoengi', 'sampledata/movie_ratings_train.pickle'), "rb"))
dtf_test = pickle.load(open(pkg_resources.resource_filename('recoengi', 'sampledata/movie_ratings_test.pickle'), "rb"))



In [3]:
M = sparse.csr_matrix((dtf_train.rating, (dtf_train.userId, dtf_train.movieId)))



In [4]:
cfm = cf.CFM(M)
cfm.computeEverything(bln_bin=False, bln_norm=True, flt_ths=2.9, ntop=64, flt_lb=-1)



DEBUG 2019-10-13 23:40:03,073 M matrix has shape 610x4180.
DEBUG 2019-10-13 23:40:03,077 B matrix has shape 610x4180.
DEBUG 2019-10-13 23:40:03,109 B matrix has sparsity 2.375872617460193%.
DEBUG 2019-10-13 23:40:03,113 Computing the similarity matrix S ...
DEBUG 2019-10-13 23:40:03,171 S matrix has shape 610x610.
DEBUG 2019-10-13 23:40:03,172 Computing the matrix SNORMALIZED ...
DEBUG 2019-10-13 23:40:03,176 Computing the matrix SCORES ...
DEBUG 2019-10-13 23:40:03,302 SCORES matrix has shape 610x4180.
DEBUG 2019-10-13 23:40:03,303 Computing the matrix AMOUNTS ...
DEBUG 2019-10-13 23:40:03,418 AMOUNTS matrix has shape 610x4180.
DEBUG 2019-10-13 23:40:03,419 Computing the performances ...
DEBUG 2019-10-13 23:40:03,774 Average global scores difference: 0.051796963793511015.
DEBUG 2019-10-13 23:40:03,776 Average positive scores difference: 0.7475288069081828.
WARNING 2019-10-13 23:40:03,777 The scores model could not perform better than a random model! Further tests needed!
DEBUG 2019-10

In [5]:
dtf_pred = pd.DataFrame(pd.Series(dict(cfm.SCORES.todok().items()))).reset_index(drop=False)
dtf_pred.columns = ["userId", "movieId", "predicted_score"]
dtf_train = pd.merge(dtf_train, dtf_pred, on=["userId", "movieId"], how="inner")
dtf_test = pd.merge(dtf_test, dtf_pred, on=["userId", "movieId"], how="inner")



In [6]:
fpr, tpr, thresholds = metrics.roc_curve(y_true=(dtf_train.rating>2.9)+1, y_score=dtf_train.predicted_score, pos_label=2)
print("AUC on training set: " + str(metrics.auc(fpr, tpr)) + ".")
fpr, tpr, thresholds = metrics.roc_curve(y_true=(dtf_test.rating>2.9)+1, y_score=dtf_test.predicted_score, pos_label=2)
print("AUC on test set: " + str(metrics.auc(fpr, tpr)) + ".")

AUC on training set: 0.6546203010067251.
AUC on test set: 0.6440409695830971.
